In [ ]:
import numpy as np
np.random.seed(1000)
from scipy import io
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# Download and save the dataset to your drive and put its path in the below function
d=(io.loadmat('/content/drive/My Drive/dataset/parsed_P0'+str(1)+'T.mat'))['RawEEGData']
l=(io.loadmat('/content/drive/My Drive/dataset/parsed_P0'+str(1)+'T.mat'))['Labels']
for j in range(2,9):
  d2=(io.loadmat('/content/drive/My Drive/dataset/parsed_P0'+str(j)+'T.mat'))['RawEEGData']
  l2=(io.loadmat('/content/drive/My Drive/dataset/parsed_P0'+str(j)+'T.mat'))['Labels']
  d=np.concatenate((d,d2),axis=0)
  l=np.concatenate((l,l2),axis=0)


k=l
k.resize(640,)
Y=k
X=d

encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)
print(Y.shape)

print(X.shape)

X_train_mat = X
y_train_mat = Y


(640, 12, 4096)
(640, 1)
(640, 12, 4096)
(640,)
(640,)
(640, 12, 4096)


In [ ]:
#Loading test data
d=(io.loadmat('/content/drive/My Drive/dataset/parsed_P0'+str(9)+'E.mat'))['RawEEGData']
for j in range(10,11):
  d2=(io.loadmat('/content/drive/My Drive/dataset/parsed_P'+str(j)+'E.mat'))['RawEEGData']
  d=np.concatenate((d,d2),axis=0)
print(d.shape)

X=d
print(X.shape)
X_test_mat = X

(80, 12, 4096)
(80, 12, 4096)


In [ ]:
y_train = y_train_mat.reshape(-1, 1)


var_list = []
for i in range(X_train_mat.shape[0]):
    var_count = X_train_mat[i].shape[-1]
    var_list.append(var_count)

var_list = np.array(var_list)
max_nb_timesteps = var_list.max()
min_nb_timesteps = var_list.min()
median_nb_timesteps = np.median(var_list)

print('max nb timesteps train : ', max_nb_timesteps)
print('min nb timesteps train : ', min_nb_timesteps)
print('median_nb_timesteps nb timesteps train : ', median_nb_timesteps)

X_train = np.zeros((X_train_mat.shape[0], X_train_mat[0].shape[0], max_nb_timesteps))

# pad ending with zeros to get numpy arrays
for i in range(X_train_mat.shape[0]):
    var_count = X_train_mat[i].shape[-1]
    #print(i, X_train_mat[i])
    X_train[i, :, :var_count] = X_train_mat[i]

# ''' Load test set '''

X_test = np.zeros((X_test_mat.shape[0], X_test_mat[0].shape[0], max_nb_timesteps))

# pad ending with zeros to get numpy arrays
for i in range(X_test_mat.shape[0]):
    var_count = X_test_mat[i].shape[-1]
    X_test[i, :, :var_count] = X_test_mat[i][:, :max_nb_timesteps]


# ''' Save the datasets '''
print("Train dataset : ", X_train.shape, y_train.shape)
print("Test dataset : ", X_test.shape)
print("Train dataset metrics : ", X_train.mean(), X_train.std())
print("Test dataset : ", X_test.mean(), X_test.std())
print("Nb classes : ", len(np.unique(y_train)))

np.save('X_train.npy', X_train)
np.save('y_train.npy', y_train)
np.save('X_test.npy', X_test)

max nb timesteps train :  4096
min nb timesteps train :  4096
median_nb_timesteps nb timesteps train :  4096.0
Train dataset :  (640, 12, 4096) (640, 1)
Test dataset :  (80, 12, 4096)
Train dataset metrics :  -6.638233682815069e-06 79.22496224741778
Test dataset :  0.0016919634029616479 49.42873207672468
Nb classes :  2


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('seaborn-paper')

from sklearn.preprocessing import LabelEncoder

import warnings
warnings.simplefilter('ignore', category=DeprecationWarning)

from keras.models import Model
from keras.layers import Permute
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K

def load_dataset_at(fold_index=None, normalize_timeseries=False, verbose=True) -> (np.array, np.array):
    if verbose: print("Loading train / test dataset : ")

    if fold_index is None:
        x_train_path = "X_train.npy"
        y_train_path = "y_train.npy"
        x_test_path =  "X_test.npy"
    
    if os.path.exists(x_train_path):
        X_train = np.load(x_train_path)
        y_train = np.load(y_train_path)
        X_test = np.load(x_test_path)
    elif os.path.exists(x_train_path[1:]):
        X_train = np.load(x_train_path[1:])
        y_train = np.load(y_train_path[1:])
        X_test = np.load(x_test_path[1:])
    else:
        raise FileNotFoundError('File %s not found!' % (TRAIN_FILES[index]))

    is_timeseries = True

    nb_classes = len(np.unique(y_train))
    y_train = (y_train - y_train.min()) / (y_train.max() - y_train.min()) * (nb_classes - 1)

    if is_timeseries:
        # scale the values
        if normalize_timeseries:
            X_train_mean = X_train.mean()
            X_train_std = X_train.std()
            X_train = (X_train - X_train_mean) / (X_train_std + 1e-8)

    if verbose: print("Finished processing train dataset..")




    if is_timeseries:
        # scale the values
        if normalize_timeseries:
            X_test = (X_test - X_train_mean) / (X_train_std + 1e-8)

    if verbose:
        print("Finished loading test dataset..")
        print()
        print("Number of train samples : ", X_train.shape[0], "Number of test samples : ", X_test.shape[0])
        print("Number of classes : ", nb_classes)
        print("Sequence length : ", X_train.shape[-1])

    return X_train, y_train, X_test, is_timeseries

def calculate_dataset_metrics(X_train):
    max_nb_variables = X_train.shape[1]
    max_timesteps = X_train.shape[-1]

    return max_timesteps, max_nb_variables



def train_model(model:Model, dataset_fold_id=None, epochs=50, batch_size=128, val_subset=None,
                cutoff=None, normalize_timeseries=False, learning_rate=1e-3, monitor='loss', optimization_mode='auto', compile_model=True):
    X_train, y_train, X_test, is_timeseries = load_dataset_at(
                                                                      fold_index=dataset_fold_id,
                                                                      normalize_timeseries=normalize_timeseries)
    max_timesteps, max_nb_variables = calculate_dataset_metrics(X_train)

    classes = np.unique(y_train)
    le = LabelEncoder()
    y_ind = le.fit_transform(y_train.ravel())
    recip_freq = len(y_train) / (len(le.classes_) *
                           np.bincount(y_ind).astype(np.float64))
    class_weight = recip_freq[le.transform(classes)]

    print("Class weights : ", class_weight)

    y_train = to_categorical(y_train, len(np.unique(y_train)))

    if is_timeseries:
        factor = 1. / np.cbrt(2)
    else:
        factor = 1. / np.sqrt(2)


    optm = Adam(lr=learning_rate)

    if compile_model:
        model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])

    if val_subset is not None:
        X_test = X_test[:val_subset]
    
    print('X_test shape and X_train shape:',X_test.shape,X_train.shape)

    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
        class_weight=class_weight, verbose=2)




Using TensorFlow backend.


In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, multiply, concatenate, Activation, Masking, Reshape
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout

DATASET_INDEX = 13

MAX_TIMESTEPS = 4096
MAX_NB_VARIABLES = 12
NB_CLASS = 2

TRAINABLE = True


def generate_model():
    ip = Input(shape=(MAX_NB_VARIABLES, MAX_TIMESTEPS))
    x = Masking()(ip)
    x = LSTM(8)(x)
    x = Dropout(0.8)(x)
    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)
    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)
    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = GlobalAveragePooling1D()(y)
    x = concatenate([x, y])
    out = Dense(NB_CLASS, activation='softmax')(x)
    model = Model(ip, out)
    model.summary()

    return model



def squeeze_excite_block(input):

    filters = input._keras_shape[-1] 

    se = GlobalAveragePooling1D()(input)
    se = Reshape((1, filters))(se)
    se = Dense(filters // 16,  activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    se = multiply([input, se])
    return se



model = generate_model()

train_model(model, epochs=500, batch_size=128)


x shape: (None, 8)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12, 4096)     0                                            
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 4096, 12)     0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 4096, 128)    12416       permute_1[0][0]                  
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 4096, 128)    512         conv1d_1[0][0]                   
_________________________________________________________________________

In [ ]:
y_pred=model.predict(X_test)
print(y_pred)
print(y_pred.shape)


[[9.88613129e-01 1.13868946e-02]
 [9.94832754e-01 5.16725797e-03]
 [9.99915004e-01 8.49969292e-05]
 [8.00084829e-01 1.99915126e-01]
 [9.99003589e-01 9.96483373e-04]
 [9.99420404e-01 5.79588173e-04]
 [9.99998927e-01 1.13231738e-06]
 [9.99986053e-01 1.39817475e-05]
 [9.99999166e-01 8.81896995e-07]
 [9.99997377e-01 2.67550854e-06]
 [9.99925971e-01 7.40529358e-05]
 [5.87271690e-01 4.12728280e-01]
 [9.98135328e-01 1.86461722e-03]
 [9.99978423e-01 2.15208529e-05]
 [9.72731650e-01 2.72683296e-02]
 [1.14676927e-03 9.98853207e-01]
 [6.78965152e-05 9.99932051e-01]
 [4.25854349e-04 9.99574125e-01]
 [5.20646691e-01 4.79353309e-01]
 [3.90142505e-03 9.96098518e-01]
 [4.58134018e-04 9.99541879e-01]
 [2.00712144e-01 7.99287915e-01]
 [1.11632096e-02 9.88836765e-01]
 [7.27104843e-02 9.27289546e-01]
 [5.63477337e-01 4.36522633e-01]
 [9.99999404e-01 6.10138159e-07]
 [9.99998450e-01 1.60597517e-06]
 [1.00000000e+00 7.03213945e-11]
 [1.00000000e+00 4.12167023e-10]
 [1.00000000e+00 1.64040365e-10]
 [1.000000

In [ ]:
y_pred = (y_pred > 0.5)
op = np.argmax(y_pred, axis=1)
for i in range(80):
  if i==40:
    print('--------')
  print(op[i])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
--------
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
for i in range(80):
  if op[i]==0:
    op[i]=1
  else:
    op[i]=2

for i in range(80):
  if i==40:
    print('--------')
  print(op[i])

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
1
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
--------
1
1
1
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
